In [1]:
using EzXML

In [2]:
include("../../src/JuliaMBD.jl")

Main.JuliaMBD

In [37]:
using JSON

In [3]:
using .JuliaMBD

In [43]:
function mkmxCell(; id::Int=-1, style::String="", parent::Int=-1, type::Symbol = :none)
    node = ElementNode("mxCell")
    if style != ""
        link!(node, AttributeNode("style", style))
    end
    if id >= 0
        link!(node, AttributeNode("id", string(id)))
    end
    if parent >= 0
        link!(node, AttributeNode("parent", string(parent)))
    end
    if type == :vertex
        link!(node, AttributeNode("vertex", "1"))
    end
    node
end

function mkmxGeometry(; x::Float64 = 0.0, y::Float64 = 0.0, width::Float64, height::Float64)
    node = ElementNode("mxGeometry")
    if x != 0.0
        link!(node, AttributeNode("x", string(x)))
    end
    if y != 0.0
        link!(node, AttributeNode("y", string(y)))
    end
    link!(node, AttributeNode("width", string(width)))
    link!(node, AttributeNode("height", string(height)))
    link!(node, AttributeNode("as", "geometry"))
    node
end

function mkuserblock(; block::Symbol, id::Int, parent::Int, width::Float64=90.0, height::Float64=60.0)
    node = ElementNode("object")
    link!(node, AttributeNode("label", "%block%"))
    link!(node, AttributeNode("block", string(block)))
    link!(node, AttributeNode("type", "block"))
    link!(node, AttributeNode("name", ""))
    link!(node, AttributeNode("placeholders", "1"))
    link!(node, AttributeNode("id", string(id)))
    tmp = mkmxCell(
        parent=parent,
        style="rounded=0;whiteSpace=wrap;html=1;fontSize=11;points=[[0,0.5,0,0,0],[1,0.5,0,0,0]];metaEdit=1;snapToPoint=0;resizable=1;rotatable=0;allowArrows=0;container=1;resizeHeight=0;connectable=0;collapsible=0;movable=1;",
        type=:vertex
    )
    link!(tmp, mkmxGeometry(width=width, height=height))
    link!(node, tmp)
    node
end

function mkinport(; name::Symbol, id::Int, parent::Int, width::Float64=10.0, height::Float64=10.0, x=-5.0, y=25.0)
    node = ElementNode("object")
    link!(node, AttributeNode("label", ""))
    link!(node, AttributeNode("type", "inport"))
    link!(node, AttributeNode("name", string(name)))
    link!(node, AttributeNode("id", string(id)))
    tmp = mkmxCell(
        parent=parent,
        style="whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[0,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;",
        type=:vertex
    )
    link!(tmp, mkmxGeometry(width=width, height=height, x=x, y=y))
    link!(node, tmp)
    node
end

function mkoutport(; name::Symbol, id::Int, parent::Int, width::Float64=10.0, height::Float64=10.0, x=85.0, y=25.0)
    node = ElementNode("object")
    link!(node, AttributeNode("label", ""))
    link!(node, AttributeNode("type", "outport"))
    link!(node, AttributeNode("name", string(name)))
    link!(node, AttributeNode("id", string(id)))
    tmp = mkmxCell(
        parent=parent,
        style="whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[1,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;fillColor=#000000;",
        type=:vertex
    )
    link!(tmp, mkmxGeometry(width=width, height=height, x=x, y=y))
    link!(node, tmp)
    node
end

function addxmlparams(node, k)
    link!(node, AttributeNode(string(k), ""))
end

function getsymbol(p, env)
    for (k,x) = env
        if p == x
            return k
        end
    end
    return :none
end
    
function mkxmlblock(b::JuliaMBD.AbstractBlock, width::Float64=90.0, height::Float64=60.0)
    id = 2
    xmlb = mkuserblock(; block=b.name, id=id, parent=1, width=width, height=height)
    id += 1
    xmlinports = []
    y = collect(range(0.0, height, length(b.inports)+2))
    for (i,p) = enumerate(b.inports)
        k = getsymbol(p, b.env)
        push!(xmlinports, mkinport(name=k, id=id, parent=2, y=y[i+1]-5))
        id += 1
    end
    xmloutports = []
    y = collect(range(0.0, height, length(b.outports)+2))
    for (i,p) = enumerate(b.outports)
        k = getsymbol(p, b.env)
        push!(xmloutports, mkoutport(name=k, id=id, parent=2, y=y[i+1]-5))
        id += 1
    end

    for p = b.parameterports
        k = getsymbol(p, b.env)
        addxmlparams(xmlb, k)
    end
    mxgraph = ElementNode("mxGraphModel")
    root = ElementNode("root")
    link!(mxgraph, root)
    link!(root, mkmxCell(id=0))
    link!(root, mkmxCell(id=1, parent=0))
    link!(root, xmlb)
    for x = xmlinports
        link!(root, x)
    end
    for x = xmloutports
        link!(root, x)
    end
    io = IOBuffer()
    print(io, mxgraph)
    str = String(take!(io)) |> x->replace(x, "<"=>"&lt;") |> x->replace(x, ">"=>"&gt;")
    
    data = Dict()
    data["xml"] = str
    data["w"] = width
    data["h"] = height
    data["aspect"] = "fixed"
    data["title"] = string(b.name)
    "<mxlibrary>[$(JSON.json(data))]</mxlibrary>"
end

mkxmlblock (generic function with 3 methods)

In [45]:
"<html>" |> x->replace(x, "<"=>"&lt;") |> x->replace(x, ">"=>"&gt;")

"&lt;html&gt;"

In [22]:
@model test begin
    @parameter begin
        b
        a = 1
    end
    @block begin
        in1 = Inport(:x)
        in2 = Inport(:y)
        out1 = Outport(:z)
    end
end

test (generic function with 1 method)

In [44]:
println(mkxmlblock(test(b=1)))

<mxlibrary>[{"xml":"&lt;mxGraphModel&gt;&lt;root&gt;&lt;mxCell id=\"0\"/&gt;&lt;mxCell id=\"1\" parent=\"0\"/&gt;&lt;object label=\"%block%\" block=\"test\" type=\"block\" name=\"\" placeholders=\"1\" id=\"2\" b=\"\" a=\"\"&gt;&lt;mxCell style=\"rounded=0;whiteSpace=wrap;html=1;fontSize=11;points=[[0,0.5,0,0,0],[1,0.5,0,0,0]];metaEdit=1;snapToPoint=0;resizable=1;rotatable=0;allowArrows=0;container=1;resizeHeight=0;connectable=0;collapsible=0;movable=1;\" parent=\"1\" vertex=\"1\"&gt;&lt;mxGeometry width=\"90.0\" height=\"60.0\" as=\"geometry\"/&gt;&lt;/mxCell&gt;&lt;/object&gt;&lt;object label=\"\" type=\"inport\" name=\"x\" id=\"3\"&gt;&lt;mxCell style=\"whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[0,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;\" parent=\"2\" vertex=\"1\"&gt;&lt;mxGeometry x=\"-5.0\" y=\"15.0\" width=\"10.0\" height=\"10.0\" as=\"geometry\"/&gt;&lt;/mxCell&gt;&lt;/objec

In [26]:
{
    "xml": "&lt;mxGraphModel&gt;&lt;root&gt;&lt;mxCell id=\"0\"/&gt;&lt;mxCell id=\"1\" parent=\"0\"/&gt;&lt;object label=\"%block%\" block=\"UserDefined\" type=\"block\" name=\"\" placeholders=\"1\" id=\"2\"&gt;&lt;mxCell style=\"rounded=0;whiteSpace=wrap;html=1;fontSize=11;points=[[0,0.5,0,0,0],[1,0.5,0,0,0]];metaEdit=1;snapToPoint=0;resizable=1;rotatable=0;allowArrows=0;container=1;resizeHeight=0;connectable=0;collapsible=0;movable=1;\" vertex=\"1\" parent=\"1\"&gt;&lt;mxGeometry width=\"90\" height=\"60\" as=\"geometry\"/&gt;&lt;/mxCell&gt;&lt;/object&gt;&lt;object label=\"&amp;lt;p style=&amp;quot;line-height: 100%;&amp;quot;&amp;gt;&amp;lt;br&amp;gt;&amp;lt;/p&amp;gt;\" type=\"inport\" name=\"in\" id=\"3\"&gt;&lt;mxCell style=\"whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[0,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;direction=east;\" vertex=\"1\" parent=\"2\"&gt;&lt;mxGeometry x=\"-5.42857142857143\" y=\"25\" width=\"10\" height=\"10\" as=\"geometry\"/&gt;&lt;/mxCell&gt;&lt;/object&gt;&lt;object label=\"&amp;lt;p style=&amp;quot;line-height: 100%;&amp;quot;&amp;gt;&amp;lt;br&amp;gt;&amp;lt;/p&amp;gt;\" type=\"outport\" name=\"out\" id=\"4\"&gt;&lt;mxCell style=\"whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[1,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;fillColor=#000000;\" vertex=\"1\" parent=\"2\"&gt;&lt;mxGeometry x=\"84.57142857142858\" y=\"25\" width=\"10\" height=\"10\" as=\"geometry\"/&gt;&lt;/mxCell&gt;&lt;/object&gt;&lt;/root&gt;&lt;/mxGraphModel&gt;",
    "w": 90,
    "h": 60,
    "aspect": "fixed",
    "title": "User-defined block"
  },

"<html>\\n"

In [29]:
unescape_string("<html>\n")

"<html>\n"

In [2]:
"""
<mxGraphModel>
  <root>
    <mxCell id=\"0\"/>
    <mxCell id=\"1\" parent=\"0\"/>
    <object label=\"%block%\" block=\"UserDefined\" type=\"block\" name=\"\" placeholders=\"1\" id=\"2\">
      <mxCell style=\"rounded=0;whiteSpace=wrap;html=1;fontSize=11;points=[[0,0.5,0,0,0],[1,0.5,0,0,0]];metaEdit=1;snapToPoint=0;resizable=1;rotatable=0;allowArrows=0;container=1;resizeHeight=0;connectable=0;collapsible=0;movable=1;\" vertex=\"1\" parent=\"1\">
        <mxGeometry width=\"90\" height=\"60\" as=\"geometry\"/>
      </mxCell>
    </object>
    <object label=\"&lt;p style=&quot;line-height: 100%;&quot;&gt;&lt;br&gt;&lt;/p&gt;\" type=\"inport\" name=\"in\" id=\"3\">
      <mxCell style=\"whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[0,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;direction=east;\" vertex=\"1\" parent=\"2\">
        <mxGeometry x=\"-5.42857142857143\" y=\"25\" width=\"10\" height=\"10\" as=\"geometry\"/>
      </mxCell>
    </object>
    <object label=\"&lt;p style=&quot;line-height: 100%;&quot;&gt;&lt;br&gt;&lt;/p&gt;\" type=\"outport\" name=\"out\" id=\"4\">
      <mxCell style=\"whiteSpace=wrap;html=1;aspect=fixed;snapToPoint=1;resizable=0;metaEdit=1;points=[[1,0.5,0,0,0]];editable=1;movable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;fillColor=#000000;\" vertex=\"1\" parent=\"2\">
        <mxGeometry x=\"84.57142857142858\" y=\"25\" width=\"10\" height=\"10\" as=\"geometry\"/>
      </mxCell>
    </object>
  </root>
</mxGraphModel>
"""

"<mxGraphModel>\n  <root>\n    <mxCell id=\"0\"/>\n    <mxCell id=\"1\" parent=\"0\"/>\n    <object label=\"%block%\" block=\"UserDefined\" type=\"block\" name=\"\" placeholders=\"1\" id=\"2\">\n      <mxCell style=\"rounded=0;whiteSpace=wrap;html=1;fontSize=11;points=[[0,0.5,0,0,0],[1,0.5,0,0,0" ⋯ 984 bytes ⋯ "ovable=1;rotatable=1;deletable=1;locked=0;connectable=1;noLabel=0;overflow=visible;fillColor=#000000;\" vertex=\"1\" parent=\"2\">\n        <mxGeometry x=\"84.57142857142858\" y=\"25\" width=\"10\" height=\"10\" as=\"geometry\"/>\n      </mxCell>\n    </object>\n  </root>\n</mxGraphModel>\n"

mkoutport (generic function with 1 method)

In [30]:
print(stdout, mkuserblock(block=:Gain, id=2, parent=1))

<object label="%block%" block="Gain" type="block" name="" placeholders="1" id="2"><mxCell style="rounded=0;whiteSpace=wrap;html=1;fontSize=11;points=[[0,0.5,0,0,0],[1,0.5,0,0,0]];metaEdit=1;snapToPoint=0;resizable=1;rotatable=0;allowArrows=0;container=1;resizeHeight=0;connectable=0;collapsible=0;movable=1;" id="2" parent="1" vertex="1"><mxGeometry width="90" height="60" as="geometry"/></mxCell></object>

In [3]:
node = ElementNode("mxGraphModel")

EzXML.Node(<ELEMENT_NODE[mxGraphModel]@0x0000aaaaf399fff0>)

In [4]:
link!(node, ElementNode("root"))

EzXML.Node(<ELEMENT_NODE[root]@0x0000aaaaf2f7cb40>)

In [5]:
node

EzXML.Node(<ELEMENT_NODE[mxGraphModel]@0x0000aaaaf399fff0>)

In [10]:
doc = XMLDocument()
setroot!(doc, node)

EzXML.Node(<ELEMENT_NODE[mxGraphModel]@0x0000aaaaf399fff0>)

In [9]:
typeof(doc)

EzXML.Node

In [11]:
write("test.xml", doc)

76